In [1]:
!pip install psycopg2-binary sqlalchemy pandas

In [2]:
from sqlalchemy import create_engine
import pandas as pd

In [3]:
host = 'psinderpostgres.postgres.database.azure.com'
port = '5432'
database = 'psinder-db'
user = 'readonly_user_psinder'
password = 'cixfo6-fyqjyQ-nohbog'

connection_string = f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}'

engine = create_engine(connection_string)

In [4]:
# User profiles of those who exclusively own pet profiles of type Dog, including all details of the user accounts
query = """
select ua.*
from pet_profile pp
     join animal_type at on pp.type_id = at.type_id
     join user_account ua on pp.user_id = ua.user_id
where at.type_name = 'Dog'
except
select ua.*
from pet_profile pp
     join animal_type at on pp.type_id = at.type_id
     join user_account ua on pp.user_id = ua.user_id
where at.type_name != 'Dog';
"""

df = pd.read_sql(query, engine)

df.head()

,user_id,code_id,gender_id,nickname,email,phone_number,date_of_birth,firstname,lastname,account_description,password_hash,salt,created_datetime,is_active
0,7,31,2,Josef4cats,josja_smith@gmail.com,602123456,1988-08-20,Josef,Smith,I’m the official servant to my dog. Looking fo...,c5d9b52ac8dfc9baf3d19e8cfc4a96b2d8b9c0e1f2a3b4...,P9xZkW2eT7oLQ3mJ5yOg,2024-10-10 22:19:00,True


In [5]:
# User profiles of those who own pets from all available dog breeds, including all details of the user accounts
query = """
select *
from user_account ua
where not exists (
    select *
    from breed b
    where b.type_id = (select type_id from animal_type where type_name = 'Dog')
      and not exists (
          select *
          from pet_profile pp
          where pp.user_id = ua.user_id
            and pp.breed_id = b.breed_id
      )
);
"""

df = pd.read_sql(query, engine)

df.head()

,user_id,code_id,gender_id,nickname,email,phone_number,date_of_birth,firstname,lastname,account_description,password_hash,salt,created_datetime,is_active
0,2,7,2,margo96,margaret1996@seznam.cz,789635112,1996-06-23,Margaret,Hamadej,I am a professional breeder with a well-establ...,a1e2b3c4d5f6789012a3b4c5d6e7f8901a2b3c4d5e6f78...,R8kYpW5eJ2oLxN9uOm2zRg,2024-01-01 00:11:10,True


In [6]:
# Total number of given and received likes for each pet, including the pet profile ID, pet name,
# the count of likes given by the pet's owner, and the count of likes received by the pet
query = """
select pp.profile_id, pp.pet_name,
       count(distinct l1.profile_target_id) as given_likes,
       count(distinct l2.profile_initiator_id) as received_likes
from pet_profile pp
left join pet_like l1 on pp.profile_id = l1.profile_initiator_id
left join pet_like l2 on pp.profile_id = l2.profile_target_id
group by pp.profile_id, pp.pet_name;
"""

df = pd.read_sql(query, engine)

df.head()

,profile_id,pet_name,given_likes,received_likes
0,1,Snusicha,1,1
1,2,Tulupchik,1,0
2,3,Kozjavka,0,0
3,4,Bobik,0,0
4,5,Ajkosik,0,0


In [7]:
# Conversations with the total count of messages in each conversation, including the conversation ID and the number of messages
query = """
select c.conversation_id,
       (select count(*) from message m where m.conversation_id = c.conversation_id) as messages_count
from conversation c;
"""

df = pd.read_sql(query, engine)

df.head()

,conversation_id,messages_count
0,1,4
1,2,4
2,3,4
3,4,4
4,5,4


In [8]:
# Users who have given a grade of 10 to others and the total number of 10s they have given, including their nickname, first name, last name, and the count of given grades of 10
query = """
with users_who_gave_10 as (
  select user_giver_id
  from user_grade
  where grade = 10
),
all_users as (
  select user_id from user_account
),
g10_counts as (
  select user_giver_id, count(*) as total_given_10s
  from user_grade
  where grade = 10
  group by user_giver_id
)
select ua.nickname, ua.firstname, ua.lastname, g10_counts.total_given_10s
from user_account ua
join g10_counts on ua.user_id = g10_counts.user_giver_id
where ua.user_id in (
    (select user_id from all_users)
    intersect
    (select user_giver_id from users_who_gave_10)
);
"""

df = pd.read_sql(query, engine)

df.head()

,nickname,firstname,lastname,total_given_10s
0,mashkerz,Marija,None,1
1,margo96,Margaret,Hamadej,2
2,Josef4cats,Josef,Smith,1
3,sarah_connor,Sarah,Connor,1
4,rahul6sharma,Rahul,None,3


In [9]:
# Which pet types are present or absent in each country, including the country name, a list of present pet types,
# and a list of absent pet types based on existing pet profiles and all possible country/pet type combinations
query = """
with all_country_pet_type as (
    select c.country_id, a.type_id
    from country c
    cross join animal_type a
),
existing_country_pet_type as (
    select co.country_id, pp.type_id
    from pet_profile pp
    join city ci on pp.city_id = ci.city_id
    join country co on ci.country_id = co.country_id
    group by co.country_id, pp.type_id
),
country_type_presence as (
    select
        coalesce(c.country_name, c2.country_name) as country_name,
        coalesce(a.type_name, a2.type_name) as type_name,
        case when e.country_id is not null then 'present' else 'absent' end as presence
    from all_country_pet_type ac
    full join existing_country_pet_type e
      on ac.country_id = e.country_id
     and ac.type_id = e.type_id
    left join country c on ac.country_id = c.country_id
    left join animal_type a on ac.type_id = a.type_id
    left join country c2 on e.country_id = c2.country_id
    left join animal_type a2 on e.type_id = a2.type_id
)
select country_name,
       string_agg(type_name, ', ' order by type_name) filter (where presence = 'present') as present_types,
       string_agg(type_name, ', ' order by type_name) filter (where presence = 'absent') as absent_types
from country_type_presence
group by country_name
order by country_name;
"""

df = pd.read_sql(query, engine)

df.head()

,country_name,present_types,absent_types
0,Austria,None,"Bird, Cat, Dog, Fish, Hamster, Horse, Other, P..."
1,Belarus,Dog,"Bird, Cat, Fish, Hamster, Horse, Other, Parrot..."
2,Belgium,None,"Bird, Cat, Dog, Fish, Hamster, Horse, Other, P..."
3,Bulgaria,"Dog, Fish","Bird, Cat, Hamster, Horse, Other, Parrot, Rabb..."
4,Croatia,Cat,"Bird, Dog, Fish, Hamster, Horse, Other, Parrot..."


In [10]:
# Countries with an average pet price greater than 1500, including the country name and the average pet price rounded to two decimal places
query = """
select co.country_name, round(avg(pp.price), 2) as avg_pet_price
from country co
join city ci on co.country_id = ci.country_id
join pet_profile pp on ci.city_id = pp.city_id
where pp.price is not null
group by co.country_name
having avg(pp.price) > 1500
order by avg_pet_price desc;
"""

df = pd.read_sql(query, engine)

df.head()

,country_name,avg_pet_price
0,France,1720.00
1,United Kingdom,1639.17
2,Czechia,1561.54


In [11]:
# Pet profiles that have received at least one like, including all profile details of the pets
query = """
select pp.*
from pet_profile pp
join
(
    (select profile_id from pet_profile)
    intersect
    (select profile_target_id from pet_like)
) as test_query on test_query.profile_id = pp.profile_id;
"""

df = pd.read_sql(query, engine)

df.head()

,profile_id,gender_id,city_id,type_id,user_id,pet_name,date_of_birth,breed_id,price,certification_url,profile_description,purebred_percentage,created_datetime
0,29,2,139,4,8,Oparysh,2023-01-10,30,40,None,A lively Roborovski Hamster who loves running ...,NaN,2024-11-02 09:00:00
1,6,2,31,1,2,Pusicka,2021-06-15,1,1000,https://psinder.com/certification/pets/pusicka...,"Pusicka is a playful and loyal companion, perf...",90.0,2024-01-01 12:11:00
2,31,2,21,2,9,Zoja,2021-05-20,11,1500,https://psinder.com/certification/pets/zoja_9.url,A beautiful Persian cat with a luxurious coat....,100.0,2024-11-10 10:00:00
3,22,2,156,4,5,Nibbles,2023-05-10,28,30,None,An energetic Syrian hamster who loves explorin...,NaN,2024-09-01 06:00:00
4,16,1,31,1,2,Chasecka,2022-04-15,10,2500,None,Chasecka is a charming and affectionate King C...,100.0,2024-11-01 08:23:02


In [12]:
# Pet profiles with likes exchanged between pets whose owners are from the same country, including pet IDs, names, and their respective countries
query = """
select pl.profile_initiator_id as giver_pet_id, pp_giver.pet_name as giver_pet_name, co_giver.country_name as giver_country,
       pl.profile_target_id as receiver_pet_id, pp.pet_name as receiver_pet_name, co_target.country_name as receiver_country
from pet_profile pp
join pet_like pl on pp.profile_id = pl.profile_target_id
join pet_profile pp_giver on pl.profile_initiator_id = pp_giver.profile_id
join city ci_target on pp.city_id = ci_target.city_id
join country co_target on ci_target.country_id = co_target.country_id
join city ci_giver on pp_giver.city_id = ci_giver.city_id
join country co_giver on ci_giver.country_id = co_giver.country_id
where co_target.country_id = co_giver.country_id;
"""

df = pd.read_sql(query, engine)

df.head()

,giver_pet_id,giver_pet_name,giver_country,receiver_pet_id,receiver_pet_name,receiver_country
0,1,Snusicha,Czechia,16,Chasecka,Czechia
1,7,Skibidi,United Kingdom,26,PiDidi,United Kingdom
2,16,Chasecka,Czechia,1,Snusicha,Czechia
3,20,Cleopatra,Russia,24,Baron,Russia
4,24,Baron,Russia,20,Cleopatra,Russia


In [13]:
# Pairs of matched pet profiles and their owners with common languages shared by the owners,
# including details such as pet names, genders, breed, type, owner information, and shared languages
query = """
with pair_data as (
    select
        pm.profile_id_1, pm.profile_id_2,
        p1.pet_name as pet_name_1, p2.pet_name as pet_name_2,
        g1.gender_name as pet_gender_1, g2.gender_name as pet_gender_2,
        b.breed_name,
        at.type_name,
        u1.user_id as owner_id_1, u1.firstname as owner_firstname_1, u1.nickname as owner_nickname_1,
        u2.user_id as owner_id_2, u2.firstname as owner_firstname_2, u2.nickname as owner_nickname_2
    from pet_match pm
    join pet_profile p1 on pm.profile_id_1 = p1.profile_id
    join pet_profile p2 on pm.profile_id_2 = p2.profile_id
    join gender g1 on p1.gender_id = g1.gender_id
    join gender g2 on p2.gender_id = g2.gender_id
    join breed b on p1.breed_id = b.breed_id
    join animal_type at on p1.type_id = at.type_id
    join user_account u1 on p1.user_id = u1.user_id
    join user_account u2 on p2.user_id = u2.user_id
),
common_lang as (
    select pd.profile_id_1, pd.profile_id_2, l.language_name
    from pair_data pd
    join user_language ul1 on ul1.user_id = pd.owner_id_1
    join user_language ul2 on ul2.user_id = pd.owner_id_2 and ul1.language_id = ul2.language_id
    join language l on l.language_id = ul1.language_id
)
select pd.profile_id_1, pd.pet_name_1, pd.pet_gender_1,
       pd.owner_firstname_1, pd.owner_nickname_1,
       pd.profile_id_2, pd.pet_name_2, pd.pet_gender_2,
       pd.owner_firstname_2, pd.owner_nickname_2,
       pd.breed_name,
       pd.type_name,
    coalesce(string_agg(cl.language_name, ', '), 'no common languages') as common_languages
from pair_data pd
left join common_lang cl on cl.profile_id_1 = pd.profile_id_1 and cl.profile_id_2 = pd.profile_id_2
group by pd.profile_id_1, pd.pet_name_1, pd.pet_gender_1, pd.owner_firstname_1, pd.owner_nickname_1,
         pd.profile_id_2, pd.pet_name_2, pd.pet_gender_2, pd.owner_firstname_2, pd.owner_nickname_2,
         pd.breed_name, pd.type_name
order by pd.breed_name, pd.type_name, pd.owner_nickname_1, pd.owner_nickname_2;
"""

df = pd.read_sql(query, engine)

df.head()

,profile_id_1,pet_name_1,pet_gender_1,owner_firstname_1,owner_nickname_1,profile_id_2,pet_name_2,pet_gender_2,owner_firstname_2,owner_nickname_2,breed_name,type_name,common_languages
0,17,Goldie,Male,Max,max4ever,23,Nesushka,Female,Pat,pat_kim,Goldfish,Fish,no common languages
1,1,Snusicha,Female,Marija,mashkerz,16,Chasecka,Male,Margaret,margo96,King Charles Spaniel,Dog,"English, Czech"
2,7,Skibidi,Female,Margaret,margo96,26,PiDidi,Male,Josef,Josef4cats,Labrador Retriever,Dog,English
3,6,Pusicka,Female,Margaret,margo96,26,PiDidi,Male,Josef,Josef4cats,Labrador Retriever,Dog,English
4,21,Simba,Male,Chris,kittens101,31,Zoja,Female,Rahul,rahul6sharma,Persian,Cat,no common languages


In [14]:
# Users who have blocked others and have pets living in the same city as the users they blocked,
# including user IDs, nicknames, and city names for both the giver and receiver
query = """
select distinct ub.user_giver_id, ub.user_receiver_id,
                ua_giver.nickname as giver_nickname, ua_receiver.nickname as receiver_nickname,
                c_giver.city_name as giver_city, c_receiver.city_name as receiver_city
from user_block ub
left join user_account ua_giver on ub.user_giver_id = ua_giver.user_id
left join user_account ua_receiver on ub.user_receiver_id = ua_receiver.user_id
left join pet_profile pp_giver on pp_giver.user_id = ua_giver.user_id
left join pet_profile pp_receiver on pp_receiver.user_id = ua_receiver.user_id
join city c_giver on pp_giver.city_id = c_giver.city_id
join city c_receiver on pp_receiver.city_id = c_receiver.city_id
where pp_giver.city_id = pp_receiver.city_id;
"""

df = pd.read_sql(query, engine)

df.head()

,user_giver_id,user_receiver_id,giver_nickname,receiver_nickname,giver_city,receiver_city
0,1,3,mashkerz,max4ever,Prague,Prague
1,3,2,max4ever,margo96,Prague,Prague


In [15]:
# Users and their pets who have never received any likes, including the user ID, nickname, pet profile ID, and pet name
query = """
select distinct ua.user_id, ua.nickname, pp.profile_id, pp.pet_name
from user_account ua
join pet_profile pp on ua.user_id = pp.user_id
where not exists (
    select 1
    from pet_like pl
    where pl.profile_target_id = pp.profile_id
);
"""

df = pd.read_sql(query, engine)

df.head()

,user_id,nickname,profile_id,pet_name
0,2,margo96,14,Chempion
1,9,rahul6sharma,32,Akulina
2,8,sarah_connor,28,Kesha
3,9,rahul6sharma,30,Cherchil
4,2,margo96,10,Sigma


In [16]:
# Users who either own at least one Persian cat or have given a grade of 10 to another user, including their user IDs and nicknames
query = """
(
  select ua.user_id, ua.nickname
  from user_account ua
  join pet_profile pp on ua.user_id = pp.user_id
  join animal_type at on pp.type_id = at.type_id
  join breed b on pp.breed_id = b.breed_id
  where at.type_name = 'Cat'
    and b.breed_name = 'Persian'
)
union
(
  select ua.user_id, ua.nickname
  from user_account ua
  join user_grade ug on ua.user_id = ug.user_giver_id
  where ug.grade = 10
);
"""

df = pd.read_sql(query, engine)

df.head()

,user_id,nickname
0,1,mashkerz
1,2,margo96
2,4,kittens101
3,5,pat_kim
4,7,Josef4cats


In [17]:
# Cat profiles along with their names, city they live in, breed and gender
query = """
select pet_name, gender_name, city_name, breed_name
from (
    select pp.pet_name, ci.city_name, br.breed_name, g.gender_name, br.type_id
    from pet_profile pp
        join gender g using (gender_id)
        join city ci using (city_id)
        join breed br using (breed_id)
) as temp
join animal_type at using (type_id)
where type_name = 'Cat';
"""

df = pd.read_sql(query, engine)

df.head()

,pet_name,gender_name,city_name,breed_name
0,Tulupchik,Male,Prague,Ragdoll
1,Kozjavka,Male,Prague,Munchkin
2,Bobik,Male,Prague,Maine Coon
3,Ajkosik,Male,Prague,Exotic Shorthair
4,Murzik,Female,Paris,Siamese


In [18]:
# Top 5 most popular breeds by the number of pets, including the breed name and the total count of pets for each breed
query = """
select b.breed_name, count(*) as pet_count
from pet_profile pp
join breed b on pp.breed_id = b.breed_id
group by b.breed_name
order by pet_count desc
limit 5;
"""
# add type name
df = pd.read_sql(query, engine)

df.head()

,breed_name,pet_count
0,Persian,5
1,Labrador Retriever,4
2,Goldfish,3
3,Scottish Fold,2
4,Syrian Hamster,2


In [19]:
# Potential pairs of pets of the same breed and animal type whose owners share at least one common language,
# including pet IDs, pet names, breed, animal type, and the shared language names
query = """
select distinct p1.profile_id as profile_id_1, p1.pet_name as pet_name_1, b1.breed_name as pet_breed_1, at1.type_name as pet_type_1,
                p2.profile_id as profile_id_2, p2.pet_name as pet_name_2, b2.breed_name as pet_breed_2, at2.type_name as pet_type_2,
                string_agg(l.language_name, ', ') as common_languages
from pet_profile p1
join user_language ul1 on p1.user_id = ul1.user_id
join language l on ul1.language_id = l.language_id
join user_language ul2 on ul2.language_id = ul1.language_id
join pet_profile p2 on p2.user_id = ul2.user_id
join breed b1 on p1.breed_id = b1.breed_id
join breed b2 on p2.breed_id = b2.breed_id
join animal_type at1 on p1.type_id = at1.type_id
join animal_type at2 on p2.type_id = at2.type_id
where p1.profile_id != p2.profile_id and p1.type_id = p2.type_id and p1.breed_id = p2.breed_id
group by p1.profile_id, p1.pet_name, b1.breed_name, at1.type_name, p2.profile_id, p2.pet_name, b2.breed_name, at2.type_name
order by profile_id_1, profile_id_2;
"""

df = pd.read_sql(query, engine)

df.head()

,profile_id_1,pet_name_1,pet_breed_1,pet_type_1,profile_id_2,pet_name_2,pet_breed_2,pet_type_2,common_languages
0,1,Snusicha,King Charles Spaniel,Dog,16,Chasecka,King Charles Spaniel,Dog,"English, Czech"
1,6,Pusicka,Labrador Retriever,Dog,7,Skibidi,Labrador Retriever,Dog,"Czech, English"
2,6,Pusicka,Labrador Retriever,Dog,26,PiDidi,Labrador Retriever,Dog,English
3,7,Skibidi,Labrador Retriever,Dog,6,Pusicka,Labrador Retriever,Dog,"English, Czech"
4,7,Skibidi,Labrador Retriever,Dog,26,PiDidi,Labrador Retriever,Dog,English


In [20]:
# Average grade received by each user, including the user ID, nickname, and their average received grade. The results are sorted in descending order of the average grade
query = """
select ua.user_id, ua.nickname, round(avg(ug.grade), 2) as avg_received_grade
from user_account ua
join user_grade ug on ua.user_id = ug.user_receiver_id
group by ua.user_id, ua.nickname
order by avg_received_grade desc;
"""

df = pd.read_sql(query, engine)

df.head()

,user_id,nickname,avg_received_grade
0,9,rahul6sharma,10.00
1,2,margo96,9.67
2,6,777katya777,8.00
3,4,kittens101,8.00
4,7,Josef4cats,8.00


In [21]:
# Countries with the highest concentration of dogs, including the country name, the number of dogs, the total number of pets,
# and the percentage of dogs relative to all pets in each country. The results are sorted in descending order of the percentage of dogs
query = """
with total_pets_per_country as (
    select co.country_id, count(*)::numeric as total_pets
    from pet_profile pp
    join city ci on pp.city_id = ci.city_id
    join country co on ci.country_id = co.country_id
    group by co.country_id
),
type_pets_per_country as (
    select co.country_id, count(*)::numeric as animal_type_count
    from pet_profile pp
    join city ci on pp.city_id = ci.city_id
    join country co on ci.country_id = co.country_id
    join animal_type at on pp.type_id = at.type_id
    where at.type_name = 'Dog'
    group by co.country_id
)
select c.country_name, tp.animal_type_count, tot.total_pets,
       round((tp.animal_type_count / tot.total_pets) * 100, 2) as percentage_type
from type_pets_per_country tp
join total_pets_per_country tot on tp.country_id = tot.country_id
join country c on c.country_id = tp.country_id
order by percentage_type desc nulls last;
"""

df = pd.read_sql(query, engine)

df.head()

,country_name,animal_type_count,total_pets,percentage_type
0,Belarus,2.0,2.0,100.00
1,United Kingdom,5.0,6.0,83.33
2,Bulgaria,2.0,3.0,66.67
3,Czechia,7.0,13.0,53.85


In [22]:
# Сities that host multiple animal types, including the city name and the number of distinct animal types hosted in each city.
# Only cities with two or more distinct animal types are included in the results
query = """
select ci.city_name, count(distinct at.type_id) as distinct_animal_types
from pet_profile pp
join city ci on pp.city_id = ci.city_id
join animal_type at on pp.type_id = at.type_id
group by ci.city_name
having count(distinct at.type_id) >= 2
order by distinct_animal_types desc, ci.city_name;
"""

df = pd.read_sql(query, engine)

df.head()

,city_name,distinct_animal_types
0,Prague,3
1,Tashkent,2
2,Varna,2


In [23]:
engine.dispose()